# Data table transformation cookbook

In [ ]:
import pandas as pd
import io
import os
import webbrowser
from openai import OpenAI
from IPython.display import display

In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not change
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
# Cookbooks specific variables

# jinko folder id you want your data to be uploaded in
FOLDER_ID = "84b38c2d-9720-4305-9161-fe4e5aa05817"
# data table to transform
SOURCE_FILE_NAME = "invalid_data"
# OPTIONAL: Post the data table on jinko and get it afterwards
POST_DATA_TABLE = True

resources_dir = os.path.normpath(
    "./resources/data_table_transformation")
source_file = os.path.join(resources_dir, SOURCE_FILE_NAME + ".csv")

In [ ]:
# Utils functions


def run_code_on_dataframe(df, transform_data):
    """
    Executes a transformation function (in that case, it's provided by the LLM) on a DataFrame.

    Args:
        df: A pandas DataFrame on which the transformation is to be applied.
        transform_data: A function that takes a DataFrame as input and returns
                        a transformed DataFrame or any result.

    Returns:
        The result of the transformation function if successful, otherwise an error message.
    """
    try:
        result = transform_data(df)
        return result
    except Exception as e:
        error_message = f"An error occurred: {e}"
        print(error_message)
        return error_message


def post_data_table(source_file):
    """
    Post the data table to jinko.

    Args:
        source_file (str): Path to the CSV file containing the data table.

    Returns:
        tuple: (data_table_core_item_id, data_table_snapshot_id)
    """
    encoded_data_table = jinko.data_table_to_sqlite(source_file)
    # https://doc.jinko.ai/api/#/paths/core-v2-data_table_manager-data_table/post
    response = jinko.make_request(
        path="/core/v2/data_table_manager/data_table",
        method="POST",
        json={
            "mappings": [],
            "rawData": encoded_data_table,
        },
        options={
            "name": SOURCE_FILE_NAME,
            "folder_id": FOLDER_ID,
        },
    )
    response_json = response.json()
    data_table_core_item_id = response_json.get("coreItemId")
    data_table_snapshot_id = response_json.get("snapshotId")
    data_table_url = jinko.get_project_item_url_by_core_item_id(
        data_table_core_item_id)
    webbrowser.open(data_table_url)
    return (data_table_core_item_id, data_table_snapshot_id)


def load_data_table(data_table_core_item_id, data_table_snapshot_id):
    """
    Load a data table snapshot from Jinko and return it as a pandas DataFrame.

    Args:
        data_table_core_item_id (str): The core item ID of the data table.
        data_table_snapshot_id (str): The snapshot ID of the data table.

    Returns:
        DataFrame: A pandas DataFrame containing the data table, with the "rowId"
        column removed.
    """

    r = jinko.make_request(
        f"/core/v2/data_table_manager/data_table/{data_table_core_item_id}/snapshots/{data_table_snapshot_id}/export",
        method="POST",
        options={"output_format": "text/csv"},
    )
    csv_data = r.text
    df = pd.read_csv(io.StringIO(csv_data)).drop(columns=["rowId"])
    return df


def generate_dropped_columns(df, scoring_grouped):
    """
    Generate a DataFrame that contains info about which columns from the original DataFrame were dropped

    Parameters
    ----------
    df : pd.DataFrame
        The original DataFrame
    scoring_grouped : pd.DataFrame
        The DataFrame grouped by the scoring group

    Returns
    -------
    dropped_columns_df : pd.DataFrame
        A DataFrame with the following columns:
        raw_columns : str
            The name of the column in the original DataFrame
        dropped : bool
            Whether the column was dropped or not
        type : str
            The type of the column in the original DataFrame
    """
    raw_columns = set(df.columns)
    scoring_grouped_obs_ids = set(scoring_grouped["obsId"])
    scoring_grouped_columns = set(scoring_grouped.columns)
    all_posible_values = scoring_grouped_obs_ids.union(scoring_grouped_columns)
    dropped_column_data = []
    for col in raw_columns:
        dtype_str = str(df[col].dtype)
        if col in all_posible_values:
            dropped_column_data.append(
                {"raw_columns": col, "dropped": False, "type": dtype_str}
            )
        else:
            dropped_column_data.append(
                {"raw_columns": col, "dropped": True, "type": dtype_str}
            )
    dropped_columns_df = pd.DataFrame(dropped_column_data)
    return dropped_columns_df


def generate_dropped_observables(observable_names, scoring_grouped):
    """
    Generate a DataFrame indicating which observables were dropped.

    Parameters
    ----------
    observable_names : pd.DataFrame
        A DataFrame containing observable IDs and their types.
    scoring_grouped : pd.DataFrame
        A DataFrame containing grouped scoring data with observable IDs.

    Returns
    -------
    dropped_obs_id_df : pd.DataFrame
        A DataFrame with columns:
        obsId : str
            The ID of the observable.
        dropped : bool
            Whether the observable was dropped (True) or not (False).
        type : str
            The data type of the observable.
    """

    observable_ids = set(observable_names["obsId"])
    scoring_grouped_obs_ids = set(scoring_grouped["obsId"])
    dropped_obs_id_data = []
    for obs_id in observable_ids:
        dtype_str = str(
            observable_names[observable_names["obsId"]
                             == obs_id]["type"].values[0]
        )
        if obs_id in scoring_grouped_obs_ids:
            dropped_obs_id_data.append(
                {"obsId": obs_id, "dropped": False, "type": dtype_str}
            )
        else:
            dropped_obs_id_data.append(
                {"obsId": obs_id, "dropped": True, "type": dtype_str}
            )
    dropped_obs_id_df = pd.DataFrame(dropped_obs_id_data)
    return dropped_obs_id_df

### Step 1 : Data table loading

In [ ]:
if POST_DATA_TABLE:
    data_table_core_item_id, data_table_snapshot_id = post_data_table(
        source_file)
    df = load_data_table(data_table_core_item_id, data_table_snapshot_id)
else:
    df = pd.read_csv(source_file)

### Step 2 : prompt generation

In [ ]:
system_message = f"""
You are a Python expert.

Please help to create the function `transform_data` to process and structure the raw data. Do not provide a script, just the function `transform_data`. Your response should ONLY be based on the given context and follow the response guidelines and format instructions.

# Response Guidelines

1. If the provided context is sufficient, generate the `transform_data` function to follow the required steps
2. If the provided context is insufficient, do not generate the Python function. Instead, briefly explain why it cannot be generated and request the missing information.
3. Use the most relevant information from the raw data table, summary data and meta data.
4. Format the Python query before responding.
5. Ensure the Python function is syntactically and logically correct before finalizing.
6. Follow the exact order of table creation as described below.
7. Use real column names from the raw data table. If column names are missing, request clarification instead of assuming placeholder names.
8. Pay close attention to the sections starting with “IMPORTANT:”
9. Do not include data in the response, I only want the function transform_data

# Response Format

## If Python Generation is Successful

  - Formatted Python code
  - the output of the python function The `transform_data` function must return the dictionary outlined below:
\\`\\`\\`python
{{ "observable_names":<observable_names_df>
, "transposed_table":<transposed_table_df>
, "scoring_grouped":<scoring_grouped_df>}}
\\`\\`\\`

## If Python Generation is Not Possible
  - A brief, human-readable explanation of why the query cannot be generated.
  - A request for any missing information from the user.

# Format instructions

IMPORTANT: When generating the code, follow the exact sequence of instructions outlined below: 

## Observable table
List all observables (i.e., patient characteristics) from the raw data table. 
The `observable_names` table schema is:

observable_names (
  obsId TEXT, -- identify the list of observables from a single column (the column contains observable names) or from several columns (one per observable). 
   type TEXT -- data type by observable
)

IMPORTANT: Follow these recommendations when building the table `observable_names`:

1. ONLY for the construction of the table `observable_names`: Include both numeric and non-numeric observables INCLUDING variables such as: gender (F/M), tumor grade (A, B, C, etc.), phenotype (blue, red, etc.), genotype (wild, mutant, etc.), serostatus (positive, negative, etc.), and response to treatment (PR, partial response, etc.).
2. Do not exclude categorical data at this stage.


## Transposed table
This table transforms the raw data into a structured format by organizing observables.
The `transposed_table` table schema is:

transposed_table (
    patientId TEXT,  -- OPTIONAL: Include if patient identifiers exist in the raw data.
   obsId TEXT, -- Extracted from the table `observable_names`. 
   time TEXT, -- Represents time durations. It is a concatenation of the TIME UNIT in TEXT and the TIME VALUE in NUMERIC. ISO-8601 duration format (e.g., "P3Y6M4DT12H30M5S"). 
   value NUMERIC, -- can be REAL or INTEGER. Observable values can be drawn from multiple data columns or from a single column if observables are listed separately. 
   arm TEXT, -- OPTIONAL: Usually a string like placebo, treated, or the name of a treatment or groups (group 1, group 2, group 3) or numbers (1, 2, 3)
   narrowRangeLowBound NUMERIC, -- Can be REAL, INTEGER or NULL. ONLY if available in the raw tabular data otherwise NULL
   narrowRangeHighBound NUMERIC -- Can be REAL, INTEGER or NULL. ONLY if available in the raw tabular data otherwise NULL
)

IMPORTANT: You must take into account when creating your Python request the following elements:
1. For the TIME VALUES, if the TIME UNIT (in the same column or in another column indicating the time units) or the TIME VALUE is missing THEN NULL. IF not NULL then USE capital letters P, Y, M, W, D, T, H, M, and S as designators for EACH of the duration elements. For example, "P3Y6M4DT12H30M5S" represents a duration of "three years, six months, four days, twelve hours, thirty minutes, and five seconds". See: https://en.wikipedia.org/wiki/ISO_8601#Durations
2. EXCLUDE NON-NUMERIC OBSERVABLES, including categorical data coded as numeric but with only a few distinct values (e.g., binary (0,1) or tumor stages (1,2,3,4), which are not considered observables).
3. In the column “VALUE”, check for a mix of formats, and any cell value (each cell) that is not exclusively numeric or contains a missing data representation (e.g., NULL, NA, n.a., empty cell, and other language-specific codes) must be considered a missing value and converted to NULL.
4. DO NOT DROP ROWS with missing values in this table. So, missing values must be retained in `transposed_table` but will be ignored during aggregation in `scoring_grouped` table.
5. If a column contains only None or null values, you can drop it

## scoring_grouped table
If the "arm" column exists and is not null, group the data by "arm", "time", and "obsId". Otherwise, group by "time" and "obsId".

scoring_grouped (
   arm TEXT -- OPTIONAL, 
   time TEXT, -- the format is ISO-8601
   obsId TEXT,
   value REAL, -- the reduction function is by default the mean of values. If the user specifies another reduction function use the specified one instead.
   narrowRangeLowBound NUMERIC, -- Can be REAL, INTEGER or NULL. Default is the min of values. If the user specifies another reduction function, use the specified one instead. 
   narrowRangeHighBound NUMERIC --Can be REAL, INTEGER or NULL. Default max of values. If the user specifies another reduction function, use the specified one instead.
)

IMPORTANT: Follow these recommendations when building the 'scoring_grouped' table:
1. Create a new table `scoring_grouped` based on the `transposed_table`.
2.  This new table is a grouping of patients. The grouping SHOULD BE DONE FOR THE TRIPLET (“arm”, ”time”, “obsId”) unless otherwise specified by the user. If `arm` doesn't exist or if its value is null, the aggregation will be done on the pairs (“time”, “obsId”). Use the reduction function by default and if the user specifies another reduction function, use the specified one instead.
3. For each aggregation, ignore rows with missing “time” values OR  missing observable values (NULL values).
4. If the "narrowRangeLowBound" and "narrowRangeHighBound" columns exist in the `transposed_table`, use the existing ones instead of calculating new values.
5. Order the values of the `scoring_grouped` table in ascending order by the triplet (“arm”, “time”, “obsId”): first by the “time” column if it exists, then the “arm” column if it exists, and finally by the “obsId” column.
6. missing time values should be stored as NULL in `transposed_table` and only ignored in `scoring_grouped`. 

# Question

Generate a series of Python queries to transform the raw data table into a structured set of tables. Follow the steps in the specified order:

1. observable_names
2. transposed_table
3. scoring_grouped


# Final Consistency Check
Before finalizing, ensure:

1. The function follows the order of operations:
observable_names → transposed_table → scoring_grouped
2. No syntax errors or logical inconsistencies are present.
3. The output tables are consistent with each other.
4. A message is included confirming successful Python generation.


IMPORTANT: Ensure the generated Python function is free of syntax errors and logically consistent before finalizing the response!

# Final Instruction to Assistant
- If the Python function is successfully generated:
  - return only the function transform_data, with no surrounding text or formatting and especially no ```python surronding
  - do not include any messages indicating success.


- If the Python function cannot be generated, provide a brief explanation and request missing details.
"""

In [ ]:
# Generate a summary of the DataFrame
summary = df.describe()
head = df.head()
data_types = df.dtypes
unique_values = df.nunique()
missing_values = df.isnull().sum()

# Value counts for categorical columns
value_counts = {}
for column in df.select_dtypes(include=["object"]).columns:
    value_counts[column] = df[column].value_counts()

user_message = f"""
# Data Summary for Transformation

## Data Types
```
{data_types}
```

## Unique Values
```
{unique_values}
```

## Missing Values
```
{missing_values}
```

## Value Counts
```
{value_counts}
```

## Statistical Summary
```
{summary}
```

## Example Data
```
{head}
```

# Task
Write the function `transform_data` that transforms a given data table into the specified format.
"""

print(user_message)

### Step 3: python function creation

In [ ]:
chat_history = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

openai_client = OpenAI()
MODEL = "o4-mini"

response = openai_client.responses.create(model=MODEL, input=chat_history)

code = response.output_text

In [ ]:
print(code)

### Step 4: Run the generated python function

In [ ]:
exec(code)
code_result = run_code_on_dataframe(
    df, transform_data
)  # we assume transform_data has just been executed

observable_names = code_result["observable_names"]
transposed_table = code_result["transposed_table"]
scoring_grouped = code_result["scoring_grouped"]

display("observable_names", observable_names)
display("transposed_table", transposed_table)
display("scoring_grouped", scoring_grouped)
display(
    "dropped_observables",
    generate_dropped_observables(observable_names, scoring_grouped),
)
display("dropped_columns", generate_dropped_columns(df, scoring_grouped))

### Step 5: Push the table "scoring_grouped" to jinko

In [ ]:
encoded_data_table = jinko.df_to_sqlite(scoring_grouped)


if POST_DATA_TABLE:
    response = jinko.make_request(
        path=f"/core/v2/data_table_manager/data_table/{data_table_core_item_id}",
        method="PUT",
        json={
            "mappings": [],
            "rawData": encoded_data_table,
        },
        options={
            "version_name": "transformed " + SOURCE_FILE_NAME,
        },
    )

else:

    response = jinko.make_request(
        path="/core/v2/data_table_manager/data_table",
        method="POST",
        json={
            "mappings": [],
            "rawData": encoded_data_table,
        },
        options={
            "name": "transformed " + SOURCE_FILE_NAME,
            "folder_id": FOLDER_ID,
        },
    )

# Pretty print the response
response_json = response.json()

data_table_core_item_id = response_json.get("coreItemId")
data_table_snapshot_id = response_json.get("snapshotId")

data_table_url = jinko.get_project_item_url_by_core_item_id(
    data_table_core_item_id)
webbrowser.open(data_table_url)

is_valid_for_calibration = jinko.make_request(
    f"/core/v2/data_table_manager/data_table/{data_table_core_item_id}/snapshots/{data_table_snapshot_id}"
).json()["validations"]["fitnessFunction"]
if is_valid_for_calibration:
    print("\033[92mThe data table is valid for calibration.\033[0m")

else:
    print("\033[91mThe data table is not valid for calibration.\033[0m")